In [1]:
import torch
import torchvision
import torch.optim as optim
import pytorch_lightning as pl
from torch.nn.parameter import Parameter
from torchmetrics import PeakSignalNoiseRatio
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import datetime
import csv
import os
import argparse
from models import CondEnergyModel
from energy_based_model.experiments.utils import permute, to_numpy, init_weights, Self_Energy_log, get_dataloaders
from energy_based_model.experiments.utils import gaussian_noise, sp_noise, delete_square, generate_Y0

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--name", type=str, default='specifically test')
parser.add_argument("--dataset", type=str, default="MNIST")
parser.add_argument("--train_size", type=int, default=60000)
parser.add_argument("--test_size", type=int, default=10000)
parser.add_argument("--in_channel", type=int, default=1)
parser.add_argument("--dim_feature", type=int, default=32)
parser.add_argument("--gaussian_noise", type=float, default=0.75)
parser.add_argument("--sp_noise", type=float, default=0.1)
parser.add_argument("--square_pixels", type=int, default=20)
parser.add_argument("--degradation", type=str, default='gaussian_noise')
parser.add_argument("--Y0_type", type=str, default='id')
parser.add_argument("--lr_energy_model", type=float, default=0.001)
parser.add_argument("--lr_langevin_max", type=float, default=0.001)
parser.add_argument("--lr_langevin_min", type=float, default=0.001)
parser.add_argument("--number_step_langevin", type=int, default=10)
parser.add_argument("--use_energy_sched", action="store_true", default=False)
parser.add_argument("--use_subset", action="store_true", default=False)
parser.add_argument("--use_label", type=int, default=4)
parser.add_argument("--epochs", type=int, default=100)
parser.add_argument("--batch_size", type=int, default=100)
parser.add_argument("--train_ratio", type=float, default=1.0)
parser.add_argument("--use_unpaired", action="store_true", default=False)
parser.add_argument("--regular_data", type=float, default=0.0)
parser.add_argument("--init_noise_decay", type=float, default=1.0)
parser.add_argument("--use_energy_reg", action="store_true", default=False)
parser.add_argument("--energy_reg_weight", type=float, default=0.0)
parser.add_argument("--use_energy_L2_reg", action="store_true", default=False)
parser.add_argument("--energy_L2_reg_weight", type=float, default=0.0)
args = parser.parse_args(args=[])

In [ ]:
curr_dir_path         = os.path.join(os.path.dirname(os.path.abspath(__file__)), '..', 'experiments')
name                  = args.name
dir_work              = os.path.join(curr_dir_path, name)
cuda_device           = 0
device                = torch.device(f'cuda:{cuda_device}' if torch.cuda.is_available() else 'cpu')
batch_size            = args.batch_size
number_epoch          = args.epochs
use_energy_sched      = args.use_energy_sched
sched_step_size       = 10
sched_gamma           = 0.93
lr_energy_model       = args.lr_energy_model
lr_langevin_max       = args.lr_langevin_max
lr_langevin_min       = args.lr_langevin_min
number_step_langevin  = args.number_step_langevin
regular_data          = args.regular_data
init_noise_decay      = args.init_noise_decay
use_reg               = args.use_energy_reg
reg_weight            = args.energy_reg_weight
use_L2_reg            = args.use_energy_L2_reg
L2_reg_weight         = args.energy_L2_reg_weight
add_noise             = True
use_unpaired          = args.use_unpaired
in_channel            = args.in_channel
dim_feature           = args.dim_feature
dim_output            = 1
degradation           = args.degradation
Y0_type               = args.Y0_type
sigma_noise           = args.gaussian_noise
snp_noise             = args.sp_noise
square_pixels         = args.square_pixels
seed                  = 0
list_lr_langevin      = np.linspace(lr_langevin_max, lr_langevin_min, num=number_epoch, endpoint=True)
pl.seed_everything(0)